# Description

Score response texts for PPL using a pretrained LM. Intuition is that a response that is 'generic' contains less information, which can be measured by PPL assigned to a text sequence by a probabilistic LM.

**NOTE:** best to run on a GPU if available.


### Environment details:
   - server: rattle
   - conda env: hospo_respo_bart

In [1]:
from multiprocessing_utils import parallelize_dataframe

import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None) # don't truncate cell contents
from tqdm.notebook import tqdm
tqdm.pandas()

import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# from inference import score_ppl_for_batch_sequences
import numpy as np


/mnt/storage/clwork/users/kew/INSTALLS/anaconda3/envs/respondelligent/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
# set GPU if available
# gpu = "5"
# print(torch.cuda.is_available())

# if torch.cuda.is_available():
#     os.environ["CUDA_VISIBLE_DEVICES"] = gpu
#     torch.cuda.device_count()  # print 1

In [5]:
# print(torch.cuda.current_device())

In [6]:
test = True

In [7]:
df = pd.read_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.pkl')
print(len(df))

if test:
    df = df[df['split'] == 'test']
    print(len(df))
    
print(df.columns)

500000
24736
Index(['Unnamed: 0', 'domain', 'rating', 'review_author', 'response_author',
       'review_clean', 'response_clean', 'sentiment', 'db_internal_id',
       'establishment', 'trip_id', 'trip_url', 'country', 'split',
       'score:review_response_length_ratio', 'score:response_sentence_length',
       'score:genericness_semantic_avg', 'score:genericness_length_ratio',
       'score:review_response_wmd', 'rrgen_id'],
      dtype='object')


In [17]:
# model_path = '/srv/scratch6/kew/bart/hospo_respo/en/500k/gpt2small_scorers/fwd/checkpoint-62/'
model_path = '/srv/scratch6/kew/bart/hospo_respo/en/500k/gpt2small_scorers/fwd/checkpoint-248/'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path, pad_token_id=tokenizer.eos_token_id)

In [18]:
model.eval()
# model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [19]:
def score_text_ppl_with_lm(text, tokenizer=tokenizer, model=model):
    input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True)#.to('cuda:0')
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    ppl_score = torch.exp(loss)
    # https://stackoverflow.com/questions/61988776/how-to-calculate-perplexity-for-a-language-model-using-pytorch
    return ppl_score.cpu().item()

In [20]:
print(score_text_ppl_with_lm("Thank you for your glowing review! It is a delight to hear that you enjoyed your visit. We look forward to welcoming you again in the near future."))
print(score_text_ppl_with_lm("Thank you for your great review. Our fantastic chefs do their best to cater to all kinds of dietary requirements. Often creating off menu dishes when requested. We think they do a brilliant job. We hope to be able to impress you again with our service next time you stay with us. Be sure to call us direct for the best rates available."))

4.6997175216674805
24.573169708251953


In [11]:
t = df['response_clean'].iloc[0]*20
print(t)
t = tokenizer.encode(t, return_tensors='pt', truncation=True)
print(tokenizer.batch_decode(t))
print(t.shape)
# print(score_text_ppl_with_lm(t))

Dear Sunlily6336, There is nothing quite like Christmas in the Castle, it truly does become such a magical winter wonderland, we are so glad to learn that you were able to experience that! Thank you for your kind comments and for posting this review, it truly means a lot. We are also glad to know that this was a dream come true vacation for you, being able to be part of that memory for you is something we truly appreciate. We hope you will come back to the Castle again soon! -Fairmont Banff SpringsDear Sunlily6336, There is nothing quite like Christmas in the Castle, it truly does become such a magical winter wonderland, we are so glad to learn that you were able to experience that! Thank you for your kind comments and for posting this review, it truly means a lot. We are also glad to know that this was a dream come true vacation for you, being able to be part of that memory for you is something we truly appreciate. We hope you will come back to the Castle again soon! -Fairmont Banff S

In [14]:
# def run_scorer_in_parallel(df):
#     df['score:tgt_text_ppl'] = df['response_clean'].progress_apply(lambda x: score_text_ppl_with_lm(x))
#     return df

# df = parallelize_dataframe(df, run_scorer_in_parallel, n_cores=50)

df['score:response_text_ppl'] = df['response_clean'].progress_apply(lambda x: score_text_ppl_with_lm(x))

In [15]:
df.sort_values(by=['score:response_text_ppl'], ascending=True)

,Unnamed: 0,domain,rating,review_author,response_author,review_clean,response_clean,sentiment,db_internal_id,establishment,trip_id,trip_url,country,split,score:review_response_length_ratio,score:response_sentence_length,score:genericness_semantic_avg,score:genericness_length_ratio,score:review_response_wmd,rrgen_id,score:response_text_ppl
392518,2609721,Hotel,5,scoutdreams,InterConTampa,"Across from Mall ---SEP--- From the check-in to the check-out, we were very pleased with our stay. Parking is free in the parking garage in the rear of the property with easy access to the lobby and room elevators. 9th floor pool offers some views and the pool area was nice but could see potential for crowding if hotel is full. With a nice mall across the street that too made for convenience. The room was quite nice with large towels, feather pillows and a comfortable mattress. Windows went from floor to ceiling though the veiws could be into someones office from the adjoining business center. I would definately stay here again when in Tampa. Felt very comfortable at this hotel and it's location.",Thank you for taking the time to share your experience with us. We look forward to welcoming you back in the near future.,2 -1 3,395078,"The Westshore Grand, A Tribute Portfolio Hotel, Tampa",87623,https://www.tripadvisor.com/Hotel_Review-g34678-or1670-d87623-Reviews-The_Westshore_Grand_A_Tribute_Portfolio_Hotel_Tampa-Tampa_Florida.html,US,train,4.0,2,1.000000,1.0,4.954702,2609721,2.619564
255684,1097795,Hotel,5,Taddeusz,GrandHotelTralee,"Excellent Hotel ---SEP--- We stayed overnight on 7 June 2012 at The Grand Hotel and found the rooms and hotel to be of the high standard that is advertised on the web. The dinner and breakfast were excellent and precisely as we ordered. The rooms were warm and comfortable and quiet and we had a good nights sleep. The staff were welcoming, cheerful and competent. We plan to return and repeat the experience and are happy to recommend it. All in all a valuable asset for Tralee.",Thank you for taking the time to write a review. We are delighted to hear that you enjoyed your stay and look forward to welcoming you back in the future.,3 -1 4,49130,The Grand Hotel Tralee,212900,https://www.tripadvisor.ie/Hotel_Review-g186613-or870-d212900-Reviews-The_Grand_Hotel_Tralee-Tralee_County_Kerry.html,IE,train,3.0,2,0.997129,0.5,4.825444,1097795,3.076608
215418,1453896,Hotel,5,Richard G,Janice B,"Superb hotel ---SEP--- Beautiful location, excellent and very well run hotel. Situated in the centre of Royal Crescent the location could not have been better, with great views of the city and an easy walk into the centre and all the tourist sites",Thank you so much for taking the time to write a review. We are so pleased you enjoyed your stay and look forward to welcoming you back in the near future.,3 -2 4,1038621,The Royal Crescent Hotel & Spa,187614,https://www.tripadvisor.co.uk/Hotel_Review-g186370-or475-d187614-Reviews-The_Royal_Crescent_Hotel_Spa-Bath_Somerset_England.html,UK,train,1.0,2,0.991653,0.5,5.388003,1453896,3.256356
312032,389941,Hotel,5,mariebrett13,Alison H,"My "" go to"" base for Cambridge visits ---SEP--- Stayed in Hotel Felix for the first time yesterday and it has become my "" go to"" base for visits to my student daughter. Modern, comfortable, spotlessly clean rooms with an extra little luxury in the form of White Company cosmetics . Friendly staff, car parking and a quiet tranquil setting. I didn't eat there so can't comment on food, but the in room tea and coffee making facilities are top notch; including coffee bags and fresh milk ( stored in minibar) The best thing about the hotel is the location, just off Huntington Road, handy for A14 and A1. So no need to brave the one way system/ ring road. Perfect for access to Girton, Fitzwilliam, Churchill and Murray Edwards Colleges and very easy access to city centre via frequent buses or a pleasant 1.5 mile stroll.",Thank you very much for your review. It is great to h

In [16]:
df['score:response_text_ppl'].describe()

count    500000.000000
mean         30.227169
std          62.865044
min           2.619564
25%          16.842829
50%          23.890131
75%          34.940752
max       34380.269531
Name: score:response_text_ppl, dtype: float64

In [17]:
# save newly computed scores
if not test:
    df['score:response_text_ppl'].to_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.response_ppl_scores.pkl')


In [18]:
new_col = pd.read_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.response_ppl_scores.pkl')
new_col == df['score:response_text_ppl']

0         True
1         True
2         True
3         True
4         True
          ... 
499995    True
499996    True
499997    True
499998    True
499999    True
Name: score:response_text_ppl, Length: 500000, dtype: bool